In [1]:
from random import randint
from math import gcd

def randuntil(lb, ub, until):
    while True:
        x = randint(lb, ub)
        if until(x): return x
        
def hexdump(xs):
    return ' '.join(f'{x:08X}' for x in xs)

def is_prime(n):
  if n == 2 or n == 3: return True
  if n < 2 or n%2 == 0: return False
  if n < 9: return True
  if n%3 == 0: return False
  r = int(n**0.5)
  # since all primes > 3 are of the form 6n ± 1
  # start with f=5 (which is prime)
  # and test f, f+2 for being prime
  # then loop by 6. 
  f = 5
  while f <= r:
    if n % f == 0: return False
    if n % (f+2) == 0: return False
    f += 6
  return True    

In [2]:
def generate_keys(p=None, q=None):
    # Wybierz losowo dwie duże liczby pierwsze
    if p is None: p = randuntil(2, 1e4-1, is_prime);                   print('p =', p)
    if q is None: q = randuntil(2, 1e4-1, is_prime);                   print('q =', q)
    assert p != q
    
    # Oblicz n
    n = p*q                                                         
    print('n =', n)
    
    # Oblicz wartość funkcji Eulera dla n
    ϕ = (p - 1)*(q - 1)
    print('ϕ =', ϕ)
    
    # Wybierz liczbę względnie pierwszą (najwiekszym wspólnym dzielnikiem jest jeden) z ϕ(n) taką że 1 < e < ϕ(n) 
    e = randuntil(3, 2**32, lambda x: is_prime(x) and gcd(x, ϕ) == 1)
    print('e =', e)
    
    # Znajdujemy liczbę d, gdzie jej różnica z odwrotnością modularną liczby e jest podzielna przez φ(n)
    d = pow(e, -1, ϕ)
    print('d =', d)

    # Zwróć klucz publiczny i klucz prywatny
    return (e, n), (d, n)

In [3]:
def cipher(key, msg):
    p, n = key
    # Szyfruj wiadomość c = x^p mod n
    return [pow(x, p, n) for x in msg]


encrypt = lambda k, m: cipher(k, bytes(m, 'utf-8'))
decrypt = lambda k, m: bytes(cipher(k, m)).decode('utf-8')

In [4]:
pub, prv = generate_keys()

p = 5527
q = 7561
n = 41789647
ϕ = 41776560
e = 2760084809
d = 5956649


In [5]:
original = 'Tajny tekst do zaszyfrowania - oryginalna wersja 1'
assert len(bytes(original, 'utf-8')) == 50
print(original)

Tajny tekst do zaszyfrowania - oryginalna wersja 1


In [6]:
secret = encrypt(pub, original)
print('length =', len(secret), '\n'+'-'*11)
print(hexdump(secret))

length = 50 
-----------
0235B886 01D78C38 015991C4 00A5013A 00466A74 02226830 026B3C4C 0048564D 0180FDC5 00FDFC70 026B3C4C 02226830 010BD879 0189EF14 02226830 00572851 01D78C38 00FDFC70 00572851 00466A74 025923C6 004D4363 0189EF14 00E276D3 01D78C38 00A5013A 01286419 01D78C38 02226830 014FFA6D 02226830 0189EF14 004D4363 00466A74 008B6427 01286419 00A5013A 01D78C38 0007D8E4 00A5013A 01D78C38 02226830 00E276D3 0048564D 004D4363 00FDFC70 015991C4 01D78C38 02226830 01EF72B5


In [7]:
recevied = decrypt(prv, secret)
assert recevied == original
print(recevied)

Tajny tekst do zaszyfrowania - oryginalna wersja 1
